In [1]:
import csv
import numpy as np
import pandas as pd

In [2]:
csvFile = open("reponse.csv", "r")
reader = csv.reader(csvFile)
list_participant = []
for item in reader:
    if item[0] not in list_participant: 
        list_participant.append(item[0])
csvFile.close()

csvFile = open("reponse.csv", "r")
reader = csv.reader(csvFile)
list1 = []
list2 = []
list3 = []
list4 = []
for item in reader:
    list1.append(item[0])
    list2.append(item[1])
    list3.append(item[3])
    list4.append(item[4])
csvFile.close()
data = {'repas': list2, 'temps': list3, 'notes': list4}
df_reponse = pd.DataFrame(data, index = list1)

In [3]:
csvFile = open("repas.csv", "r")
reader = csv.reader(csvFile)
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []
for item in reader:
    list1.append(item[0])
    list2.append(item[1])
    list3.append(item[4])
    list4.append(item[5])
    list5.append(item[6])
csvFile.close()
data = {'repas':list1, 'id': list2, 'accompagnment': list3, 'protein': list4, 'dessert': list5}
df_repas = pd.DataFrame(data)

In [4]:
def ddm(series1, series2): 
    distance1 = 0
    distance2 = 0
    for i in range(4,7): 
        distance1 += abs(int(series1[i]) - int(series2[i]))
    for i in range(11,14):
        distance2 += abs(int(series1[i]) - int(series2[i]))
    return distance1, distance2

In [5]:
def ddm2(series1, series2): 
    p = np.array([
        [0, 0, 1, 2, 3, 4, 5, 6, 7, 7], 
        [0, 0, 0, 2, 3, 4, 5, 6, 7, 7], 
        [1, 0, 0, 4, 6, 2, 7, 3, 5, 5], 
        [2, 2, 4, 0, 5, 1, 3, 7, 6, 6], 
        [3, 3, 6, 5, 0, 7, 2, 1, 4, 4], 
        [4, 4, 2, 1, 7, 0, 6, 5, 3, 3], 
        [5, 5, 7, 3, 2, 6, 0, 4, 1, 1], 
        [6, 6, 3, 7, 1, 5, 4, 0, 2, 2], 
        [7, 7, 5, 6, 4, 3, 1, 2, 0, 0], 
        [7, 7, 5, 6, 4, 3, 1, 2, 0, 0]
    ])
    v = np.array([
        [0, 7, 7, 5, 6, 4, 4, 1, 7, 6], 
        [7, 0, 4, 1, 2, 6, 7, 5, 2, 4], 
        [7, 4, 0, 4, 4, 5, 3, 7, 1, 2], 
        [5, 1, 4, 0, 4, 6, 7, 7, 4, 4], 
        [6, 2, 4, 4, 0, 7, 7, 3, 2, 1], 
        [4, 6, 5, 6, 7, 0, 2, 4, 7, 5], 
        [4, 7, 3, 7, 7, 2, 0, 4, 6, 5], 
        [1, 5, 7, 7, 3, 4, 4, 0, 5, 6], 
        [7, 2, 1, 4, 2, 7, 6, 5, 0, 4], 
        [6, 4, 2, 4, 1, 5, 5, 6, 4, 0]
    ])
    distance1 = p[int(series1[3])-1][int(series2[3])-1]
    distance2 = v[int(series1[10])-1][int(series2[10])-1]
    return distance1, distance2

In [6]:
frames = []
for i in range(0, len(list_participant)-1): 
    df_participant = df_reponse[df_reponse.index == list_participant[i]]
    df_participant1 = pd.merge(df_participant, df_repas, on = 'repas')
    df_participant1.index = df_participant.index
    df_participant2 = pd.concat([df_participant1[0:10].sort_values(by = "temps"), 
                                 df_participant1[10:20].sort_values(by = "temps")], 
                                 axis = 1)
    df_participant2.columns = ['repas_p', 'temps_p', 'notes_p', 
                               'i_p', 'a_p', 'p_p', 'd_p', 
                               'repas_v', 'temps_v', 'notes_v', 
                               'i_v', 'a_v', 'p_v', 'd_v']
    df_participant3 = pd.DataFrame([
        df_participant2.iloc[0], 
        df_participant2.iloc[2], 
        df_participant2.iloc[4], 
        df_participant2.iloc[6], 
        df_participant2.iloc[8]
    ])
    df_participant4 = pd.DataFrame([
        df_participant2.iloc[1], 
        df_participant2.iloc[3], 
        df_participant2.iloc[5], 
        df_participant2.iloc[7], 
        df_participant2.iloc[9]
    ])
    df_participant5 = pd.concat([df_participant2, df_participant3, df_participant4])
    list1 = []
    list2 = []
    for i in range(0,20):
        if i < 19:
            list1.append(ddm(df_participant5.iloc[i], df_participant5.iloc[i+1])[0])
            list2.append(ddm(df_participant5.iloc[i], df_participant5.iloc[i+1])[1])
        else: 
            list1.append(ddm(df_participant5.iloc[i], df_participant5.iloc[0])[0])
            list2.append(ddm(df_participant5.iloc[i], df_participant5.iloc[0])[1])
    df_participant5['score_p'] = list1
    df_participant5['score_v'] = list2
    df_participant6 = df_participant5.ix[:,[3,2,14,10,9,15]]
    frames.append(df_participant6)

H:\anacoda\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [7]:
result1 = pd.concat(frames)
result1[0:20]
result1.to_csv('result2.csv')

In [8]:
frames2 = []
for i in range(0, len(list_participant)-1): 
    df_participant = df_reponse[df_reponse.index == list_participant[i]]
    df_participant1 = pd.merge(df_participant, df_repas, on = 'repas')
    df_participant1.index = df_participant.index
    df_participant2 = pd.concat([df_participant1[0:10].sort_values(by = "temps"), 
                                 df_participant1[10:20].sort_values(by = "temps")], 
                                 axis = 1)
    df_participant2.columns = ['repas_p', 'temps_p', 'notes_p', 
                               'i_p', 'a_p', 'p_p', 'd_p', 
                               'repas_v', 'temps_v', 'notes_v', 
                               'i_v', 'a_v', 'p_v', 'd_v']
    df_participant3 = pd.DataFrame([
        df_participant2.iloc[0], 
        df_participant2.iloc[2], 
        df_participant2.iloc[4], 
        df_participant2.iloc[6], 
        df_participant2.iloc[8]
    ])
    df_participant4 = pd.DataFrame([
        df_participant2.iloc[1], 
        df_participant2.iloc[3], 
        df_participant2.iloc[5], 
        df_participant2.iloc[7], 
        df_participant2.iloc[9]
    ])
    df_participant5 = pd.concat([df_participant2, df_participant3, df_participant4])
    list1 = []
    list2 = []
    for i in range(0,20):
        if i < 19:
            list1.append(ddm2(df_participant5.iloc[i], df_participant5.iloc[i+1])[0])
            list2.append(ddm2(df_participant5.iloc[i], df_participant5.iloc[i+1])[1])
        else: 
            list1.append(ddm2(df_participant5.iloc[i], df_participant5.iloc[0])[0])
            list2.append(ddm2(df_participant5.iloc[i], df_participant5.iloc[0])[1])
    df_participant5['score_p'] = list1
    df_participant5['score_v'] = list2
    df_participant6 = df_participant5.ix[:,[3,2,14,10,9,15]]
    frames2.append(df_participant6)

H:\anacoda\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [9]:
result2 = pd.concat(frames2)
result2[0:20]
result2.to_csv('result3.csv')